# Harry potter story generator using GPT-2 (huggingface)

In [1]:
# Import necessary libraries
import torch
from torch import nn
import torch.nn.functional as F
import torchdata
import torchtext

from tqdm import tqdm
import random, math, time
from torch.autograd import Variable

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
# SEED = 555
# torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

cuda:0


## Data Preparation

In [2]:
# I used the data from https://github.com/ErikaJacobs/Harry-Potter-Text-Mining
import os 
path = './data' # assign directory
hp_files = [os.path.join(path, filename) for filename in os.listdir(path)]
hp_files = hp_files[0:2] # Use only two files for testing.
len(hp_files) #7 files, each file indicate each book.

2

In [3]:
# Load the data into dataframes
import pandas as pd
# Modified from the code provided by 
Book_list = list()

df = pd.read_csv(hp_files[0], sep="@")
df.head()

flag_token = 0
for book in hp_files:
    if flag_token == 0:
        df = pd.read_csv(book, sep="@")
        flag_token = 1
    else:
        df2 = pd.read_csv(book, sep="@")
        df = pd.concat([df, df2])

print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 18
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     36 non-null     object
 1   Chapter  36 non-null     int64 
 2   Book     36 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ KB
None
                                                Text  Chapter  Book
0  THE BOY WHO LIVED  Mr. and Mrs. Dursley, of nu...        1     1
1  THE VANISHING GLASS  Nearly ten years had pass...        2     1
2  THE LETTERS FROM NO ONE  The escape of the Bra...        3     1
3  THE KEEPER OF THE KEYS  BOOM. They knocked aga...        4     1
4  DIAGON ALLEY  Harry woke early the next mornin...        5     1


In [4]:
# Seperated into smaller sentences
temp_list = list()
for idx, sentence in df.iterrows():
    temp = sentence.Text.strip().lower().split()
    chunks = [" ".join(temp[x:x+30]) for x in range(0, len(temp), 30)]
    temp_list.extend(chunks)

In [5]:
# Clean dataset a bit.

from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import re

nlp = spacy.load('en_core_web_md')

def preprocessing(sentence):
    
    # Remove backslash
    sentence.replace("\\", "")

    # Clear the html tag by using regular expression.
    sentence = re.sub("<[^>]*>", "", sentence) # Filter html tag
    sentence = re.sub("[^\x00-\x7F]+", "", sentence) # Filter non-English
    sentence = re.sub("/[^a-zA-Z0-9 ]/", "", sentence) # Filter out some symbol
    #It matches any character which is not contained in the ASCII character set (0-127, i.e. 0x0 to 0x7F)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    # This time "I WILL NOT FILTERS OUT STOPWORD" during the preprocessing as I think it is
    # necessary for story. For the punctuation I think it should be fine to filter out.
    for token in doc:
        if token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM' and token.pos_!= 'X':
                cleaned_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(cleaned_tokens)

# #let's apply to the whole dataframe
# for i, row in df.iterrows():
#     clean_text = preprocessing(row.Text)
#     df.at[i, 'Clean_text'] = clean_text

data = list()
for sentence in temp_list:
     sentence_clean = preprocessing(sentence)
     data.append(sentence_clean)

2023-03-25 15:55:27.855395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 15:55:30.962345: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-25 15:55:30.962687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-25 15:55:30.962698: W tensorfl

In [6]:
from sklearn.model_selection import train_test_split

ds_train, ds_valid = train_test_split(data, test_size=0.3, random_state=555)

In [7]:
from datasets import Dataset
from datasets import DatasetDict
import pandas as pd

raw_datasets_train = Dataset.from_pandas(pd.DataFrame(data = {'content': ds_train}))
raw_datasets_valid = Dataset.from_pandas(pd.DataFrame(data = {'content': ds_valid}))

#remove .shuffle if you want to train the whole dataset....

raw_datasets = DatasetDict(
    {
        'train':raw_datasets_train,
        'valid':raw_datasets_valid
    }
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 3862
    })
    valid: Dataset({
        features: ['content'],
        num_rows: 1656
    })
})

In [8]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key][:200]}")

CONTENT: bit with a mum an dad like your what else would yeh be an i reckon it be abou time yeh read yer letter.\ harry stretch out his hand at last


## Preprocessing

In [9]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.padding_side = "right" # "left" or "right"
tokenizer.pad_token = tokenizer.eos_token

outputs = tokenizer(
    raw_datasets["train"][:2]["content"], 
    return_tensors="pt",
    padding=True,
    )


print(f"Input IDs length: {len(outputs['input_ids'])}")
print(outputs['input_ids'])

Input IDs length: 2
tensor([[ 2545,   351,   257, 25682,   281,  9955,   588,   534,   644,  2073,
           561,  9838,    71,   307,   281,  1312, 43851,   340,   307,   450,
           280,   640,  9838,    71,  1100,   331,   263,  3850,    13,    59,
          3971,   563,  7539,   503,   465,  1021,   379,   938],
        [ 1462,   883,   508,  1265,   329,   340,    13,    59,   475,   644,
           922,   307,   777,  1573,   508,  3446,   307,   484, 11691,   284,
          1265,   329,  1037,   618,  2506,   307,   655,   355, 10416,   290,
         12008,   355, 50256, 50256, 50256, 50256, 50256, 50256]])


In [10]:
def tokenize(element):
    outputs = tokenizer(
        element["content"], 
        return_tensors="pt",
        padding=True,
    )
    
    input_batch = []
    for input_ids in outputs["input_ids"]:
        input_batch.append(input_ids)
    return {"input_ids": input_batch}

# raw_datasets_ = Dataset.from_pandas(pd.DataFrame(data=raw_datasets_train))
tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 3862
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 1656
    })
})

In [11]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [12]:

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.4M parameters


In [13]:
keytoken_ids = []
for keyword in [
    "harry",
    "potter",
    "voldemort"
]:
    ids = tokenizer([keyword]).input_ids[0]
    if len(ids) == 1:
        keytoken_ids.append(ids[0])
    else:
        print(f"Keyword has not single token: {keyword}")

Keyword has not single token: harry
Keyword has not single token: potter
Keyword has not single token: voldemort


In [14]:
from torch.nn import CrossEntropyLoss
import torch

def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False) #change to reduction=None
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    # weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
    #     axis=[0, 2]
    # )
    # weights = alpha * (1.0 + weights)
    # Calculate weighted average
    # weighted_loss = (loss_per_sample * weights).mean()
    weighted_loss = loss_per_sample.mean()
    return weighted_loss

In [15]:
# print(len(tokenized_datasets['valid']['input_ids']))

In [16]:
# tokenized_datasets['valid']['input_ids'][1].shape

In [17]:
# temp = list()
# for i in range(1656):
#     if tokenized_datasets['train']['input_ids'][i].shape[0] != 62:
#         print(f"lenght = {tokenized_datasets['train']['input_ids'][i].shape[0]}")
#         print(f'index = {i}')
#         temp.append(i)

In [18]:
from torch.utils.data.dataloader import DataLoader

tokenized_datasets.set_format("torch")

#train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)

# Reduce the size otherwise, my pc will melted.

train_dataloader = DataLoader(tokenized_datasets['train']['input_ids'][0:800], batch_size=16, shuffle=True)
eval_dataloader  = DataLoader(tokenized_datasets["train"]['input_ids'][800:1000], batch_size=16)

In [19]:
weight_decay = 0.1


def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [20]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():

            outputs = model(batch["input_ids"].to(device), labels=batch["input_ids"].to(device))
            outputs.loss = outputs.loss.reshape(1)
        losses.append(accelerator.gather(outputs.loss))        
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [21]:
model = GPT2LMHeadModel(config)
model = model.to(device)

In [22]:
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(model), lr=5e-4)

In [23]:
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision='fp16')

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [24]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

In [25]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (!f()).
Your token has been saved to /root/.huggingface/token
Login successful


In [26]:
!git config --global credential.helper

!f() { /root/.vscode-server/bin/ee2b180d582a7f601fa6ecfdad8d9fd269ab1884/node /tmp/vscode-remote-containers-f9eebabf-4ff7-4b68-a8c5-6466c52d9040.js git-credential-helper $*; }; f


In [27]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "Harry_potter_story_generator"
repo_name = get_full_repo_name(model_name)
repo_name

'TonsonP/Harry_potter_story_generator'

In [28]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

output_dir = "Harry_potter_story_generator"
repo = Repository(output_dir, clone_from=repo_name)

/root/projects/NLP/Assignment/Huggingface_Story_Generator/Harry_potter_story_generator is already a clone of https://huggingface.co/TonsonP/Harry_potter_story_generator. Make sure you pull the latest changes with `repo.git_pull()`.


In [29]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():

            #outputs = model(batch["input_ids"].to(device), labels=batch["input_ids"].to(device))
            outputs = model(batch.to(device), labels=batch.to(device))
            outputs.loss = outputs.loss.reshape(1)
        losses.append(accelerator.gather(outputs.loss))        
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [30]:
evaluate()

(10.572465896606445, 39044.8359375)

In [32]:
# training
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
eval_steps = 5_000

model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=num_training_steps
    ):
        #logits = model(batch["input_ids"]).logits
        batch = batch.to(device)
        logits = model(batch).logits
        #loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        loss = keytoken_weighted_loss(batch, logits, keytoken_ids)

        if step % 100 == 0:
            accelerator.print(
                {
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        print(loss)
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % (eval_steps * gradient_accumulation_steps)) == 0:
        #if step % 1000 == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(repo_name, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(repo_name)
                repo.push_to_hub(
                    commit_message=f"Training in progress step {step}", blocking=False
                )

  0%|          | 0/50 [00:00<?, ?it/s]

tensor(1.3225, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3197, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3228, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3284, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3192, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3248, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3214, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3194, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3312, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3180, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3159, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3246, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3265, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3201, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3284, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3222, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.3031, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.2989, device='cuda:0', grad_fn=<DivBack

In [34]:
# Commit after finish training
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
tokenizer.save_pretrained(output_dir)
repo.push_to_hub(commit_message=f"Training in progress step {step}", blocking=False)

Several commits (2) will be pushed upstream.


('https://huggingface.co/TonsonP/Harry_potter_story_generator/commit/7efe5be85690ac6e5b1b7b63a43016e4c55d076e',
 [push command, status code: running, in progress. PID: 5681])

In [41]:
# I do not know why but it not save json so I need to save manually.
model.config.to_json_file("config.json")

In [45]:
model.save_pretrained('TonsonP/Harry_potter_story_generator')

In [38]:
import torch
from transformers import pipeline

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.padding_side = "right" # "left" or "right"
tokenizer.pad_token = tokenizer.eos_token


pipe = pipeline("text-generation", max_length=100, pad_token_id=0, eos_token_id=0, model='TonsonP/Harry_potter_story_generator', tokenizer=tokenizer)

In [44]:
# Not so good, but okay.
txt = "Mr.Potter"
print(pipe(txt, num_return_sequences=50)[0]["generated_text"])

Mr.Potter Drake conservation


In [41]:
# Compare with the one available on the internet
pipe2 = pipeline("text-generation", max_length=100, pad_token_id=0, eos_token_id=0, model='ceostroff/harry-potter-gpt2-fanfiction')

Downloading:   0%|          | 0.00/765 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [42]:
# Way better
txt = """Mr.Potter"""
print(pipe2(txt, num_return_sequences=50)[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Mr.Potter – the House-Elf is asking permission to use that elf's name as a way to defend himself. The other side, or the house elves, are being punished by this House as if you were the one who betrayed them. And if I say that House-Elves are being punished for any crime, I must state that no one is ever to make a mistake."It was that last phrase that Harry recognized from his father's book, and it was clear that he was
